In [1]:
import pickle
import numpy
import pandas as pd
from os import path
from os import listdir
import sys
from azureml.core import Dataset
from azureml.core import Workspace
from tempfile import TemporaryDirectory
import tempfile

state_gov_nm = 'MA'
year_to_process = '2020'
data_file_location = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/beefy/code/QuigleyData'

#state_gov_nm = sys.argv[1]

In [2]:
ws = Workspace.get(name="ml-quigley"
                   , subscription_id='8d8bcbfe-86b4-42bb-a470-a8cf97c98d69'
                   , resource_group='project-quigley')

local_ds_path = tempfile.mkdtemp()

#define the azureml dataset

leg_meta_ds = Dataset.get_by_name(ws,'leg_meta_ds')

In [3]:
leg_data = pd.DataFrame()
_cmltv_event_df = path.join(data_file_location,state_gov_nm + 'cmltv_events.df')

In [4]:
#mount the azureml dataset
if 1==1:
#with TemporaryDirectory() as local_ds_path:
    with leg_meta_ds.mount(local_ds_path):
        for f in listdir(local_ds_path):
            if state_gov_nm in f:
                _LegOverallData_CSV = path.join(local_ds_path, f)
                print(_LegOverallData_CSV)
                leg_data = leg_data.append(pd.read_csv(_LegOverallData_CSV))
                print(len(leg_data))
                print('loaded')

print(len(leg_data))

/tmp/tmpdm1xqoq4/MA2010LegOverallData.csv
26321
loaded
/tmp/tmpdm1xqoq4/MA2012LegOverallData.csv
80589
loaded
/tmp/tmpdm1xqoq4/MA2014LegOverallData.csv
127727
loaded
/tmp/tmpdm1xqoq4/MA2016LegOverallData.csv
174195
loaded
/tmp/tmpdm1xqoq4/MA2018LegOverallData.csv
224105
loaded
/tmp/tmpdm1xqoq4/MA2020LegOverallData.csv
267427
loaded
267427


In [5]:
#create a cumulative event list for each time_t
#also creation of a column to determine when a pass or fail occurs, vs a regular move (can expand on this later on)
e_hist_list = []
prev_bill_id = 0
event_hist = ''
move_list = []
first = True
#iterate through the rows of the dataframe
for r in leg_data.iterrows():
    if r[1]['bill_id'] == prev_bill_id:
        event_hist = event_hist + str(r[1]['event']) + '->'
        e_hist_list.append(event_hist)
        if 'enacted' in event_hist:
            move_list.append('Pass')
        else:
            move_list.append('Move')
    else:
        event_hist = str(r[1]['event']) + '->'
        e_hist_list.append(event_hist)
        if first:
            first = False
        elif move_list[-1] != 'Pass':
            move_list[-1] = 'Fail'
        
        move_list.append('Move')
        
    prev_bill_id = r[1]['bill_id']

In [6]:
#add the new lists as columns in the dataframe
leg_data['cmltv_event'] = e_hist_list
leg_data['move'] = move_list

In [7]:
leg_to_export = leg_data[['bill_id','time_t','cmltv_event','move']].copy()

with open(_cmltv_event_df,'wb') as f:
    pickle.dump(leg_to_export, f)